In [3]:
api_key = 'a5d04411cecb64b2e73e2bc896da553e'

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import sqlite3 as sq
import numpy as np
from sqlalchemy import create_engine
from pprint import pprint
import csv, sqlite3

# Import API key
#from api_keys import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [5]:
capitals = pd.read_csv('data/capitalscsv.csv')
capitals.head()

,Capital,State
0,Montgomery,Alabama
1,Juneau,Alaska
2,Phoenix,Arizona
3,Little Rock,Arkansas
4,Sacramento,California


In [6]:
capitals['Capital']

0         Montgomery
1             Juneau
2            Phoenix
3        Little Rock
4         Sacramento
5             Denver
6           Hartford
7              Dover
8        Tallahassee
9            Atlanta
10          Honolulu
11             Boise
12       Springfield
13      Indianapolis
14        Des Moines
15            Topeka
16         Frankfort
17       Baton Rouge
18           Augusta
19         Annapolis
20            Boston
21           Lansing
22        Saint Paul
23           Jackson
24    Jefferson City
25            Helena
26           Lincoln
27       Carson City
28           Concord
29           Trenton
30          Santa Fe
31            Albany
32           Raleigh
33          Bismarck
34          Columbus
35     Oklahoma City
36             Salem
37        Harrisburg
38        Providence
39          Columbia
40            Pierre
41         Nashville
42            Austin
43    Salt Lake City
44        Montpelier
45          Richmond
46           Olympia
47        Cha

In [7]:
#url
url = "http://api.openweathermap.org/data/2.5/weather?"

query_url = f"{url}appid={api_key}&units=imperial&q="

In [8]:
#run a test to see data and [u]
test3 = requests.get(query_url + 'New York').json()
test3
#gotta dig into dic
#response['name']
#response['clouds']['all']
#response['sys']['country']
#response['dt']
#response['main']['temp']
#response['coord']['lon']
#response['coord']['lat']
#response['main']['temp_max']
#response['wind']['speed']

{'coord': {'lon': -73.99, 'lat': 40.73},
 'weather': [{'id': 802,
   'main': 'Clouds',
   'description': 'scattered clouds',
   'icon': '03n'}],
 'base': 'stations',
 'main': {'temp': 81.73,
  'pressure': 1020,
  'humidity': 39,
  'temp_min': 77,
  'temp_max': 84.99},
 'visibility': 16093,
 'wind': {'speed': 4.7, 'deg': 200},
 'clouds': {'all': 40},
 'dt': 1561768280,
 'sys': {'type': 1,
  'id': 4686,
  'message': 0.0115,
  'country': 'US',
  'sunrise': 1561714015,
  'sunset': 1561768273},
 'timezone': -14400,
 'id': 5128581,
 'name': 'New York',
 'cod': 200}

In [9]:
capital_df = pd.DataFrame({'City' : capitals['Capital']})
capital_df['Cloudiness'] = ''
capital_df['Country'] = ''
capital_df['Date'] = ''
capital_df['Humidity'] = ''
capital_df['Lat'] = ''
capital_df['Lng'] = ''
capital_df['Max Speed'] = ''
capital_df['Wind Speed'] = ''
capital_df['Temp (F)'] = ''
row_count = 1
capital_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Speed,Wind Speed,Temp (F)
0,Montgomery,,,,,,,,,
1,Juneau,,,,,,,,,
2,Phoenix,,,,,,,,,
3,Little Rock,,,,,,,,,
4,Sacramento,,,,,,,,,


In [10]:
print('Executing Data Retrieval')
for index, row in capital_df.iterrows():
    city = row['City']
    response = requests.get(query_url + city).json()
    
    if response["cod"] == "404":
        print('City is no bueno... skipping city...')
        capital_df.loc[index, 'Country'] = 'Drop'
    else: 

        print(f' Processing Record {row_count} for {city}')

        capital_df.loc[index, 'Cloudiness'] = float(response['clouds']['all'])
        capital_df.loc[index, 'Country'] = (response['sys']['country'])
        capital_df.loc[index, 'Date'] = response['dt']
        capital_df.loc[index, 'Humidity'] = float(response['main']['humidity'])
        capital_df.loc[index, 'Lat'] = float(response['coord']['lat'])
        capital_df.loc[index, 'Lng'] = float(response['coord']['lon'])
        capital_df.loc[index, 'Max Speed'] = float(response['main']['temp_max'])
        capital_df.loc[index, 'Wind Speed'] = float(response['wind']['speed'])
        capital_df.loc[index, 'Temp (F)'] = float(response["main"]["temp"])
        row_count = row_count + 1
#        weather_df[weather_df.name != city]
#        row_count = row_count + 1

Executing Data Retrieval
 Processing Record 1 for Montgomery
 Processing Record 2 for Juneau
 Processing Record 3 for Phoenix
 Processing Record 4 for Little Rock
 Processing Record 5 for Sacramento
 Processing Record 6 for Denver
 Processing Record 7 for Hartford
 Processing Record 8 for Dover
 Processing Record 9 for Tallahassee
 Processing Record 10 for Atlanta
 Processing Record 11 for Honolulu
 Processing Record 12 for Boise
 Processing Record 13 for Springfield
 Processing Record 14 for Indianapolis
 Processing Record 15 for Des Moines
 Processing Record 16 for Topeka
 Processing Record 17 for Frankfort
 Processing Record 18 for Baton Rouge
 Processing Record 19 for Augusta
 Processing Record 20 for Annapolis
 Processing Record 21 for Boston
 Processing Record 22 for Lansing
 Processing Record 23 for Saint Paul
 Processing Record 24 for Jackson
 Processing Record 25 for Jefferson City
 Processing Record 26 for Helena
 Processing Record 27 for Lincoln
 Processing Record 28 for Car

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [11]:
#capital_df =  capital_df[capital_df.Country != 'Drop']

In [12]:
capital_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 10 columns):
City          50 non-null object
Cloudiness    50 non-null object
Country       50 non-null object
Date          50 non-null object
Humidity      50 non-null object
Lat           50 non-null object
Lng           50 non-null object
Max Speed     50 non-null object
Wind Speed    50 non-null object
Temp (F)      50 non-null object
dtypes: object(10)
memory usage: 4.0+ KB


In [13]:
capital_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Speed,Wind Speed,Temp (F)
0,Montgomery,1,US,1561768398,74,32.37,-86.3,89.01,1.45,81.97
1,Juneau,40,US,1561768583,23,58.3,-134.42,84.2,8.05,83.19
2,Phoenix,40,US,1561768504,6,33.45,-112.08,111,13.87,107.64
3,Little Rock,1,US,1561768504,78,34.75,-92.29,86,4.7,81.5
4,Sacramento,1,US,1561768580,17,38.58,-121.49,89.01,11.41,83.14
5,Denver,40,US,1561768116,16,39.74,-104.98,93.2,12.75,89.38
6,Hartford,1,US,1561768628,44,41.76,-72.69,84.2,3.36,76.75
7,Dover,1,US,1561768681,66,39.16,-75.52,84.2,4.32,79.74
8,Tallahassee,1,US,1561768629,74,30.44,-84.28,84.99,4.7,83.44
9,Atlanta,75,US,1561768628,48,33.75,-84.39,86,9.17,81.97


In [14]:
capital_df.to_csv("output_data/Capital_Weather.csv")

# SQL Connection

In [20]:
connection = "root:Bibiana14!@localhost:3306"
engine = create_engine(f'mysql://{connection}')
engine.execute('CREATE DATABASE ETL_db')
engine.execute('USE ETL_db')

ProgrammingError: (MySQLdb._exceptions.ProgrammingError) (1007, "Can't create database 'etl_db'; database exists") [SQL: 'CREATE DATABASE ETL_db'] (Background on this error at: http://sqlalche.me/e/f405)

In [95]:
capital_df.to_sql('us_capitals', con=engine, if_exists='append')

In [15]:
capital_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Speed,Wind Speed,Temp (F)
0,Montgomery,1,US,1561768398,74,32.37,-86.3,89.01,1.45,81.97
1,Juneau,40,US,1561768583,23,58.3,-134.42,84.2,8.05,83.19
2,Phoenix,40,US,1561768504,6,33.45,-112.08,111,13.87,107.64
3,Little Rock,1,US,1561768504,78,34.75,-92.29,86,4.7,81.5
4,Sacramento,1,US,1561768580,17,38.58,-121.49,89.01,11.41,83.14
5,Denver,40,US,1561768116,16,39.74,-104.98,93.2,12.75,89.38
6,Hartford,1,US,1561768628,44,41.76,-72.69,84.2,3.36,76.75
7,Dover,1,US,1561768681,66,39.16,-75.52,84.2,4.32,79.74
8,Tallahassee,1,US,1561768629,74,30.44,-84.28,84.99,4.7,83.44
9,Atlanta,75,US,1561768628,48,33.75,-84.39,86,9.17,81.97


In [33]:

data = capital_df
sql_data = 'data\capital.sqlite' #- Creates DB names SQLite
conn = sq.connect(sql_data)
cur = conn.cursor()


cur.execute('''DROP TABLE IF EXISTS SA''')
data.to_sql('Capitals', conn, if_exists='replace', index=False) # - writes the pd.df to SQLIte DB


In [34]:
pd.read_sql('select * from Capitals', conn)



,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Speed,Wind Speed,Temp (F)
0,Montgomery,1.0,US,1561768398,74.0,32.37,-86.30,89.01,1.45,81.97
1,Juneau,40.0,US,1561768583,23.0,58.30,-134.42,84.20,8.05,83.19
2,Phoenix,40.0,US,1561768504,6.0,33.45,-112.08,111.00,13.87,107.64
3,Little Rock,1.0,US,1561768504,78.0,34.75,-92.29,86.00,4.70,81.50
4,Sacramento,1.0,US,1561768580,17.0,38.58,-121.49,89.01,11.41,83.14
5,Denver,40.0,US,1561768116,16.0,39.74,-104.98,93.20,12.75,89.38
6,Hartford,1.0,US,1561768628,44.0,41.76,-72.69,84.20,3.36,76.75
7,Dover,1.0,US,1561768681,66.0,39.16,-75.52,84.20,4.32,79.74
8,Tallahassee,1.0,US,1561768629,74.0,30.44,-84.28,84.99,4.70,83.44
9,Atlanta,75.0,US,1561768628,48.0,33.75,-84.39,86.00,9.17,81.97


In [35]:
conn.commit()
conn.close()